# 06 - Model Deployment

This notebook creates a unit test component that checks if the model deployed and endpoint are working correctly.

In [ ]:
import os
import time
import logging
import kfp
from google.cloud import bigquery, storage
from google.cloud import aiplatform as vertex_ai
from google_cloud_pipeline_components.experimental.custom_job import utils
from kfp.v2 import compiler, dsl
from kfp.v2.dsl import component
from typing import NamedTuple
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output, Metrics,
                        OutputPath, component)

from google_cloud_pipeline_components.experimental.custom_job import utils

In [3]:
logging.basicConfig(level=logging.INFO)

## Load Params and Resource Config

In [4]:
from config.gcp_resource import *

In [5]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    
if SERVICE_ACCOUNT == "" or SERVICE_ACCOUNT is None or SERVICE_ACCOUNT == "[your-service-account]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.account)' 2>/dev/null
    SERVICE_ACCOUNT = shell_output[0]
    
if GCS_BUCKET == "" or GCS_BUCKET is None or GCS_BUCKET == "[your-bucket-name]":
    # Get your bucket name to GCP projet id
    GCS_BUCKET = PROJECT_ID
    # Try to create the bucket if it doesn'exists
    ! gsutil mb -l $REGION gs://$BUCKET
    print("")
    
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


In [6]:
print("Train machine type", TRAIN_COMPUTE)
print("Deploy machine type", DEPLOY_COMPUTE)
print("Deployment:", DEPLOY_IMAGE)
print('PIPELINE_ROOT: {}'.format(PIPELINE_ROOT))
print('MODULE_ROOT: {}'.format(MODULE_ROOT))
print('DATA_ROOT: {}'.format(DATA_ROOT))
print('SERVING_MODEL_DIR: {}'.format(SERVING_MODEL_DIR))

Train machine type n1-standard-4
Deploy machine type n1-standard-4
Deployment: us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest
PIPELINE_ROOT: gs://mle_airbus_dataset/airbusmlepipeline/pipeline_root
MODULE_ROOT: gs://mle_airbus_dataset/airbusmlepipeline/pipeline_module
DATA_ROOT: gs://mle_airbus_dataset/airbusmlepipeline/data
SERVING_MODEL_DIR: gs://mle_airbus_dataset/airbusmlepipeline/serving_model


## Model Deployment Unit Test

In [11]:
@component(packages_to_install=["numpy", "google-cloud-aiplatform", "google-cloud-storage", "Pillow"], output_component_file="./build/test_deployment_component.yaml",)
def test_deployment_component(
    endpoint: Input[Artifact],
    project_dict: dict
    ):
    import logging
    import numpy as np
    import google.cloud.aiplatform as aip
    from google.cloud import storage
    from io import BytesIO
    from PIL import Image   
    
    PROJECT_ID = project_dict['PROJECT_ID']
    REGION = project_dict['REGION']
    GCS_BUCKET = project_dict['GCS_BUCKET']
    aip.init(project=PROJECT_ID, location=REGION)
    logging.info(endpoint.uri)
    logging.info(endpoint.metadata)
    
    client = storage.Client() # Implicit environment set up
    bucket = client.get_bucket(GCS_BUCKET)
    fname_list = ['00b846e38.jpg', '060ea266e.jpg', '0aa565354.jpg', '00ce2c1c0.jpg', '0a286fb15.jpg']
    img_ori_list = []

    for fname in fname_list:
        blob = bucket.get_blob(f'train_v2/{fname}')
        img_big = Image.open(BytesIO(blob.download_as_bytes()))
        img_ori_list.append(img_big)
    
    endpoint = aip.Endpoint(endpoint.uri.split("/")[-1])
    
    for idx, image in enumerate(img_ori_list):
        img = image.resize((128, 128))
        input_img = (np.array(img.getdata())/255.0).reshape(128, 128, 3).tolist()
        prediction = endpoint.predict(instances=[input_img])
        result = np.array(prediction.predictions)[0]

    logging.info("Model deployment unit testing passed!")